In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import os

C:\Users\82109\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
os.chdir('C:/Users/82109/Desktop')

In [88]:
df_ns = pd.read_excel('실적데이터.xlsx')
df = pd.read_excel('Data_0914.xlsx')
df_on_test = pd.read_excel('df_on_test.xlsx')

# 간단한 전처리

In [118]:
df = df.astype({'date':'datetime64[ns]'})

In [89]:
df['test'] = df_on_test['test']
df

,date,exposure,mot_code,product_code,product_name,group,product_detail,brand,price,tot_price,...,early,month,season,day_night,group_time1,group_time2,group_exposure,detail,temperature,test
0,2019-01-01 06:00:00,20,100346,201072,테이트 남성 셀린니트3종,의류,상의,테이트,39900,2099000,...,0,1,겨울,오전,B,B,0,0,-3.70,0
1,2019-01-01 06:00:00,20,100346,201079,테이트 여성 셀린니트3종,의류,상의,테이트,39900,4371000,...,0,1,겨울,오전,B,B,0,0,-3.70,0
2,2019-01-01 06:20:00,20,100346,201072,테이트 남성 셀린니트3종,의류,상의,테이트,39900,3262000,...,0,1,겨울,오전,B,B,0,0,-3.71,0
3,2019-01-01 06:20:00,20,100346,201079,테이트 여성 셀린니트3종,의류,상의,테이트,39900,6955000,...,0,1,겨울,오전,B,B,0,0,-3.71,0
4,2019-01-01 06:40:00,20,100346,201072,테이트 남성 셀린니트3종,의류,상의,테이트,39900,6672000,...,0,1,겨울,오전,B,B,0,1,-3.71,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38295,2019-12-31 23:20:00,20,100448,201391,일시불쿠첸압력밥솥 6인용,주방,밥솥,쿠첸,148000,1664000,...,0,12,겨울,저녁,C,C,0,0,-4.94,0
38296,2019-12-31 23:40:00,20,100448,201383,무이자쿠첸압력밥솥 10인용,주방,밥솥,쿠첸,178000,9149000,...,0,12,겨울,저녁,C,C,0,1,-4.94,1
38297,2019-12-31 23:40:00,20,100448,201384,무이자쿠첸압력밥솥 6인용,주방,밥솥,쿠첸,158000,2328000,...,0,12,겨울,저녁,C,C,0,1,-4.94,0
38298,2019-12-31 23:40:00,20,100448,201390,일시불쿠첸압력밥솥 10인용,주방,밥솥,쿠첸,168000,15282000,...,0,12,겨울,저녁,C,C,0,1,-4.94,0


In [90]:
df_ns.rename(columns = {'방송일시':'date','노출(분)':'exposure'},inplace = True)

In [91]:
df_ns.fillna(method = 'ffill',inplace = True)

In [92]:
df['exposure'] = df_ns['exposure']

In [94]:
drop_ns1 = df_ns[df_ns['상품군']=="무형"].index
drop_ns2 = df_ns[df_ns['판매단가']>df_ns['취급액']].index

In [95]:
df_ns.drop(drop_ns1,inplace = True)
df_ns.drop(drop_ns2,inplace = True)

In [96]:
drop_index1 = df[df['group']=="무형"].index
drop_index2 = df[df['price']>df['tot_price']].index

In [97]:
df.drop(drop_index1,inplace = True)
df.drop(drop_index2,inplace = True)

In [98]:
df_ns = df_ns.iloc[:-4,:]

In [99]:
df_ns.columns

Index(['date', 'exposure', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '취급액'], dtype='object')

In [100]:
df['exposure'].describe()

count    35378.000000
mean        20.339275
std          3.094256
min          2.466667
25%         20.000000
50%         20.000000
75%         20.000000
max         40.000000
Name: exposure, dtype: float64

In [69]:
df

,date,exposure,mot_code,product_code,product_name,group,product_detail,brand,price,tot_price,...,early,month,season,day_night,group_time1,group_time2,group_exposure,detail,temperature,test
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,상의,테이트,39900,2099000,...,0,1,겨울,오전,B,B,0,0,-3.70,0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,상의,테이트,39900,4371000,...,0,1,겨울,오전,B,B,0,0,-3.70,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,상의,테이트,39900,3262000,...,0,1,겨울,오전,B,B,0,0,-3.71,0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,상의,테이트,39900,6955000,...,0,1,겨울,오전,B,B,0,0,-3.71,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,상의,테이트,39900,6672000,...,0,1,겨울,오전,B,B,0,1,-3.71,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38295,2019-12-31 23:20:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,밥솥,쿠첸,148000,1664000,...,0,12,겨울,저녁,C,C,0,0,-4.94,0
38296,2019-12-31 23:40:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,밥솥,쿠첸,178000,9149000,...,0,12,겨울,저녁,C,C,0,1,-4.94,1
38297,2019-12-31 23:40:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,밥솥,쿠첸,158000,2328000,...,0,12,겨울,저녁,C,C,0,1,-4.94,0
38298,2019-12-31 23:40:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,밥솥,쿠첸,168000,15282000,...,0,12,겨울,저녁,C,C,0,1,-4.94,0


# 시청률 추가

In [103]:
from datetime import datetime,timedelta
from math import ceil
from dateutil.parser import parse

In [104]:
# 시청률 데이터
df_rating = pd.read_excel('시청률데이터.xlsx')
df_rating

,시간대,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,...,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-12-30,2019-12-31,2019-01-01 to 2019-12-31
0,02:00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.003
1,02:01,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.012,0.003
2,02:02,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004
3,02:03,0.000,0.000,0.014,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.017,0.000,0.000,0.000,0.000,0.000,0.000,0.004
4,02:04,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.017,0.000,0.000,0.000,0.000,0.000,0.000,0.004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1436,01:56,0.000,0.000,0.000,0.000,0.027,0.000,0.000,0.000,0.000,...,0.013,0.000,0.000,0.000,0.015,0.000,0.000,0.000,0.000,0.004
1437,01:57,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,...,0.013,0.000,0.000,0.000,0.015,0.000,0.000,0.000,0.000,0.004
1438,01:58,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,...,0.013,0.017,0.000,0.000,0.015,0.000,0.000,0.019,0.000,0.004
1439,01:59,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,...,0.013,0.000,0.000,0.000,0.015,0.000,0.000,0.000,0.000,0.004


In [105]:
# 양쪽 맨 끝에 있는 평균값 제거하기
df_rating = df_rating.iloc[:1440]
df_rating = df_rating.iloc[:,:-1]
df_rating.set_index('시간대',inplace = True)
df_rating

,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,2019-01-10,...,2019-12-22,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-12-30,2019-12-31
시간대,,,,,,,,,,,,,,,,,,,,,
02:00,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.000,0.000,...,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.000,0.000
02:01,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.000,0.005,...,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.000,0.012
02:02,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.000,0.005,...,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.000,0.000
02:03,0.0,0.0,0.014,0.0,0.000,0.0,0.0,0.0,0.000,0.005,...,0.0,0.000,0.000,0.017,0.0,0.000,0.0,0.0,0.000,0.000
02:04,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.000,0.005,...,0.0,0.000,0.000,0.017,0.0,0.000,0.0,0.0,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
01:55,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.004,0.000,...,0.0,0.013,0.000,0.000,0.0,0.015,0.0,0.0,0.000,0.000
01:56,0.0,0.0,0.000,0.0,0.027,0.0,0.0,0.0,0.000,0.000,...,0.0,0.013,0.000,0.000,0.0,0.015,0.0,0.0,0.000,0.000
01:57,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.004,0.000,...,0.0,0.013,0.000,0.000,0.0,0.015,0.0,0.0,0.000,0.000


In [106]:
# 데이터 index와 column을 합친다
date = []
rating = []
for day in df_rating.columns:
    for time in df_rating.index:
        date.append('{} {}'.format(day,time))
        rating.append(df_rating.loc[time,day])
        
df_time = pd.DataFrame({'date':date,'rating':rating})
df_time

,date,rating
0,2019-01-01 02:00,0.0
1,2019-01-01 02:01,0.0
2,2019-01-01 02:02,0.0
3,2019-01-01 02:03,0.0
4,2019-01-01 02:04,0.0
...,...,...
525595,2019-12-31 01:55,0.0
525596,2019-12-31 01:56,0.0
525597,2019-12-31 01:57,0.0
525598,2019-12-31 01:58,0.0


In [107]:
# 빠른 실행을 위하여 0값 드랍
df_time = df_time[df_time.rating !=0]
df_time

,date,rating
9,2019-01-01 02:09,0.014
100,2019-01-01 03:40,0.014
101,2019-01-01 03:41,0.011
102,2019-01-01 03:42,0.011
103,2019-01-01 03:43,0.011
...,...,...
525497,2019-12-31 00:17,0.024
525498,2019-12-31 00:18,0.050
525499,2019-12-31 00:19,0.050
525500,2019-12-31 00:20,0.024


In [108]:
# 홈쇼핑 데이터와 시간 기준을 같게 만들고 str을 datetime으로 변경
def ys(date):
    if date[-5:]<'02:00':
        return datetime.strptime(date,'%Y-%m-%d %H:%M') + timedelta(days=1)
    else:
        return datetime.strptime(date,'%Y-%m-%d %H:%M')
df_time.date = df_time.date.apply(ys)

C:\Users\82109\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [109]:
df_time.sort_values('date')

,date,rating
9,2019-01-01 02:09:00,0.014
100,2019-01-01 03:40:00,0.014
101,2019-01-01 03:41:00,0.011
102,2019-01-01 03:42:00,0.011
103,2019-01-01 03:43:00,0.011
...,...,...
525497,2020-01-01 00:17:00,0.024
525498,2020-01-01 00:18:00,0.050
525499,2020-01-01 00:19:00,0.050
525500,2020-01-01 00:20:00,0.024


In [110]:
# date를 인덱스로 사용
df_time.set_index('date',inplace=True)

In [111]:
# 편성표에 맞추어 시청률 평균값 구하기
def rating_TV(row,df_time):
    st = str(row.date)
    fi = str(row.date + timedelta(minutes=ceil(row.exposure))) # 시간을 올림하였다.
    return( df_time[st:fi].sum())/ceil(row.exposure) # 0값을 날렸기에 더하고 기간으로 나눈다.

In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35378 entries, 0 to 38299
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            35378 non-null  object 
 1   exposure        35378 non-null  float64
 2   mot_code        35378 non-null  int64  
 3   product_code    35378 non-null  int64  
 4   product_name    35378 non-null  object 
 5   group           35378 non-null  object 
 6   product_detail  35378 non-null  object 
 7   brand           35378 non-null  object 
 8   price           35378 non-null  int64  
 9   tot_price       35378 non-null  int64  
 10  sales           35378 non-null  float64
 11  hour            35378 non-null  int64  
 12  dow             35378 non-null  object 
 13  group_dow       35378 non-null  int64  
 14  ratings         35378 non-null  float64
 15  early           35378 non-null  int64  
 16  month           35378 non-null  int64  
 17  season          35378 non-null 

In [119]:
df['rating_mean'] = df.apply(rating_TV,df_time=df_time,axis=1)

In [126]:
df.to_excel('df.xlsx',index = False)

# 평가데이터

In [4]:
test = pd.read_excel('평가데이터.xlsx')
test = test.astype({'마더코드':'object', '상품코드':'object'})
test['디테일'] = 0

In [5]:
for row in range(test.shape[0]-1):
    if (test['마더코드'][row] == test['마더코드'][row +1]) & (test['상품코드'][row] == test['상품코드'][row+1]):
        continue
    elif test['마더코드'][row] != test['마더코드'][row+1]:
        test['디테일'][row] +=1 

<ipython-input-5-7acc6c39563b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['디테일'][row] +=1


In [6]:
for row in range(test.shape[0]-1):
    if row <= 6:
        continue
    elif (test['디테일'][row] == 1) & (test['상품명'][row] == test['상품명'][row-2] and test['상품명'][row]!=test['상품명'][row-1]):
        test['디테일'][row-1] += 1
    elif (test['디테일'][row] == 1) & (test['상품명'][row] == test['상품명'][row-3]):
        test['디테일'][row-1] +=1
        test['디테일'][row-2] +=1
    elif (test['디테일'][row] == 1) & (test['상품명'][row] == test['상품명'][row-4]):
        test['디테일'][row-1] +=1
        test['디테일'][row-2] +=1
        test['디테일'][row-3] +=1
    elif (test['디테일'][row] == 1) & (test['상품명'][row] == test['상품명'][row-6]):
        test['디테일'][row-1] +=1
        test['디테일'][row-2] +=1
        test['디테일'][row-3] +=1
        test['디테일'][row-4] +=1
        test['디테일'][row-5] +=1
test

<ipython-input-6-fd2f05213b23>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['디테일'][row-1] += 1
<ipython-input-6-fd2f05213b23>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['디테일'][row-1] +=1
<ipython-input-6-fd2f05213b23>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['디테일'][row-2] +=1
<ipython-input-6-fd2f05213b23>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Se

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,디테일
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,0
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,0
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,1
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,0
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,0
...,...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,0
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,0
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,1
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN,0


In [7]:
detail_index = test[test['상품코드']==200124].index
detail_index1 = [1047,2066,2150,2751]

In [8]:
test['디테일'][detail_index] =0
test['디테일'][detail_index1] =1

<ipython-input-8-d9a4e3d02943>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['디테일'][detail_index] =0
<ipython-input-8-d9a4e3d02943>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['디테일'][detail_index1] =1


In [11]:
test['노출(분)'].fillna(method = 'ffill',inplace = True)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   방송일시    2891 non-null   datetime64[ns]
 1   노출(분)   2891 non-null   float64       
 2   마더코드    2891 non-null   object        
 3   상품코드    2891 non-null   object        
 4   상품명     2891 non-null   object        
 5   상품군     2891 non-null   object        
 6   판매단가    2891 non-null   int64         
 7   취급액     0 non-null      float64       
 8   디테일     2891 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 203.4+ KB


In [17]:
drop_detail = test.iloc[2550:2563,:].index
test['디테일'][drop_detail] = 0

<ipython-input-17-93bda313ec36>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['디테일'][drop_detail] = 0


In [19]:
drop_detail1 = test.iloc[2029:2042,:].index
test['디테일'][drop_detail1] =1

<ipython-input-19-3f3b778dc6e0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['디테일'][drop_detail1] =1


In [24]:
test['디테일'][-1:] =1
test

<ipython-input-24-0eecf891c3b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['디테일'][-1:] =1


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,디테일
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,0
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,0
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,1
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,0
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,0
...,...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,0
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,0
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,1
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN,0


In [20]:
test.to_excel('test.xlsx',index = False)